In [1]:
import spacy
import numpy as np

In [2]:
file = open(r"C:\Users\akhil\OneDrive\Desktop\Natural-Language-Processing-with-Python-master\06-Deep-Learning\moby_dick_four_chapters.txt")

In [3]:
fe = file.read()
#print(fe)

In [4]:
nlp = spacy.load('en_core_web_sm', disable=['parser','tagger','ner'])
nlp.max_length = 1198623


def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']                                 


tokens = separate_punc(fe) 
print(len(tokens))
print("tokens: ",tokens[:30])

11338
tokens:  ['call', 'me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on', 'shore', 'i', 'thought', 'i']


C:\Users\akhil\anaconda3\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [5]:
train_len = 26
text_sequences = []
for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)

for i in range(3):
    print(f"Sequence No.{i}: ",' '.join(text_sequences[i]))
print("\nLength of all sequences: ",len(text_sequences))    

Sequence No.0:  call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on
Sequence No.1:  me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore
Sequence No.2:  ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i

Length of all sequences:  11312


In [6]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)
print("Sequences: ",sequences[0])
#print("\n token Index: \n",tokenizer.index_word)

Sequences:  [956, 14, 263, 51, 261, 408, 87, 219, 129, 111, 954, 260, 50, 43, 38, 314, 7, 23, 546, 3, 150, 259, 6, 2713, 14, 24]


In [7]:
# for i in sequences[0]:
#     print(f"{i}: {tokenizer.word_counts}")

In [80]:
vocabulary_size = len(tokenizer.word_counts)
print("vocabulary_size: ",vocabulary_size)
sequences = np.array(sequences)
print(sequences)

vocabulary_size:  2718
[[ 956   14  263 ... 2713   14   24]
 [  14  263   51 ...   14   24  957]
 [ 263   51  261 ...   24  957    5]
 ...
 [ 952   12  166 ...  262   53    2]
 [  12  166 2712 ...   53    2 2718]
 [ 166 2712    3 ...    2 2718   26]]


In [107]:
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size,25,input_length = seq_len))
    model.add(LSTM(150,return_sequences = True))
    model.add(LSTM(150))
    model.add(Dense(150,activation = 'relu'))
    model.add(Dense(vocabulary_size, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    model.summary()
    
    return model

In [108]:
from keras.utils import to_categorical
print(sequences)
print("\nexcept last row:\n",sequences[:,:-1])
X = sequences[:,:-1]
y = sequences[:,-1]
y = to_categorical(y,num_classes = vocabulary_size + 1)
seq_len = X.shape[1]
print("\n\nSequential Length: ",seq_len)

[[ 956   14  263 ... 2713   14   24]
 [  14  263   51 ...   14   24  957]
 [ 263   51  261 ...   24  957    5]
 ...
 [ 952   12  166 ...  262   53    2]
 [  12  166 2712 ...   53    2 2718]
 [ 166 2712    3 ...    2 2718   26]]

except last row:
 [[ 956   14  263 ...    6 2713   14]
 [  14  263   51 ... 2713   14   24]
 [ 263   51  261 ...   14   24  957]
 ...
 [ 952   12  166 ...   11  262   53]
 [  12  166 2712 ...  262   53    2]
 [ 166 2712    3 ...   53    2 2718]]


Sequential Length:  25


In [109]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            67975     
                                                                 
 lstm (LSTM)                 (None, 25, 150)           105600    
                                                                 
 lstm_1 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 150)               22650     
                                                                 
 dense_1 (Dense)             (None, 2719)              410569    
                                                                 
Total params: 787,394
Trainable params: 787,394
Non-trainable params: 0
_________________________________________________________________


In [111]:
from pickle import dump, load

model.fit(X,y,batch_size=128,epochs = 100,verbose =1)

Epoch 1/100
89/89 [==============================] - 11s 91ms/step - loss: 6.8501 - accuracy: 0.0443
Epoch 2/100
89/89 [==============================] - 8s 95ms/step - loss: 6.3855 - accuracy: 0.0529
Epoch 3/100
89/89 [==============================] - 8s 88ms/step - loss: 6.3554 - accuracy: 0.0529
Epoch 4/100
89/89 [==============================] - 8s 89ms/step - loss: 6.2570 - accuracy: 0.0529
Epoch 5/100
89/89 [==============================] - 8s 88ms/step - loss: 6.1398 - accuracy: 0.0529
Epoch 6/100
89/89 [==============================] - 8s 89ms/step - loss: 6.0567 - accuracy: 0.0544
Epoch 7/100
89/89 [==============================] - 8s 89ms/step - loss: 5.9474 - accuracy: 0.0652
Epoch 8/100
89/89 [==============================] - 8s 88ms/step - loss: 5.8306 - accuracy: 0.0675
Epoch 9/100
89/89 [==============================] - 8s 86ms/step - loss: 5.7379 - accuracy: 0.0682
Epoch 10/100
89/89 [==============================] - 8s 88ms/step - loss: 5.6639 - accuracy: 0.072

In [112]:
model.save('NLP_LSTM_K.h5')

In [114]:
dump(tokenizer, open('NLP_LSTM_k.h5', 'wb'))